## Import Libraries

In [7]:
import matplotlib.pyplot as plt
import urllib
from bs4 import BeautifulSoup
from selenium import webdriver
from matplotlib import pyplot as plt
import re
import os,sys, shutil
import time
from datetime import date
try:
    import cPickle as pickle
except:
    import pickle
import pprint
from collections import deque
from shutil import copyfile
import random
import glob
# Import the required modules
import cv2, os
import numpy as np
from PIL import Image
from sklearn.cross_validation import KFold
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# Logistic Regression
from sklearn import datasets
from sklearn import metrics

In [8]:
pkl_fl = open("linkedin_profiles.pickle","rb")
my_original_list=pickle.load(pkl_fl) # errors out here
pkl_fl.close()

## label the Images and store into respective folders

In [3]:
directory = "Male"
    
if os.path.exists(directory):
    shutil.rmtree(directory)
    os.makedirs(directory)     
else:
    os.makedirs(directory) 

directory1 = "Female"

if os.path.exists(directory1):
    shutil.rmtree(directory1)
    os.makedirs(directory1)     
else:
    os.makedirs(directory1)     

directory2 = "Label_Images_Gender"

if os.path.exists(directory2):
    shutil.rmtree(directory2)
    os.makedirs(directory2)     
else:
    os.makedirs(directory2)     
    
fileList = glob.glob("./Images/*.*")

for id,fp in enumerate(fileList):
    filename, file_extension = os.path.splitext(fp)
    uid = filename.split('/')[-1]
    #print fp
    for prof in my_original_list:
        if prof['User_ID'] == uid:
            new_file_extension = prof['Gender']
            new_file_extension = new_file_extension.title()
            #os.rename(fp, filename+"."+new_file_extension)
            copyfile(filename+".jpg", './Label_Images_Gender/'+ uid + '.' + str(id) + "."+new_file_extension +'.jpg')
            copyfile(filename+".jpg", new_file_extension +'/' + uid + ".jpg")

In [9]:
# For face detection we will use the Haar Cascade provided by OpenCV.
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)

# For face recognition we will the the LBPH Face Recognizer 
recognizer = cv2.createLBPHFaceRecognizer()

In [10]:
def get_images_and_labels(path):
    # Append all the absolute image paths in a list image_paths
    
    image_paths = [os.path.join(path, f) for f in os.listdir(path)]
    # images will contains face images
    images = []
    # labels will contains the label that is assigned to the image
    labels = []
    #gender will contains 1 or 0 indecating male or female
    gender =[]
    
    for image_path in image_paths:
        # Read the image and convert to grayscale
        try:
            image_pil = Image.open(image_path).convert('L')
            # Convert the image format into numpy array
            image = np.array(image_pil, 'uint8')
            # Get the label of the image
        except:
            pass
        
        nbr = int(os.path.split(image_path)[1].split(".")[1])
        gender_current = os.path.split(image_path)[1].split(".")[2]
        print nbr
        
        # Detect the face in the image
        faces = faceCascade.detectMultiScale(image)
        # If face is detected, append the face to images and the label to labels
        try:
            for (x, y, w, h) in faces:

                ref_image = image[y: y + h, x: x + w]
                resized = cv2.resize(ref_image, (100, 100), interpolation = cv2.INTER_AREA)
                edge_images = cv2.Canny(resized,100,200)
                
                images.append(np.hstack((resized.reshape(1,10000), edge_images.reshape(1,10000))))
                labels.append(nbr)

                if gender_current == 'Male':
                    gender.append(1)
                else:
                    gender.append(0)

                cv2.imshow("Adding faces to traning set...", resized)
                cv2.waitKey(1)
        except:
            pass
    # return the images list and labels list
    print "lables"
    print labels
    print "gender_current"
    print gender
    
    return images, labels, gender

In [11]:
images, labels, gender = get_images_and_labels('Label_Images_Gender')

cv2.destroyAllWindows()

1870
491
2508
3052
1700
2382
132
783
1643
209
2392
442
1609
2895
406
2689
2754
89
299
1382
1096
1824
1208
1689
980
2924
2015
321
2799
1213
2802
2109
2553
2635
2175
2275
42
3192
641
260
1937
1978
2587
754
1843
1592
516
2416
3232
558
1591
2524
970
1306
135
2184
859
676
737
3118
1976
229
1904
134
2817
2317
1810
3298
2927
3036
1282
2462
1350
196
659
1108
1561
371
2664
2094
704
2352
73
1140
2783
580
786
2593
855
1154
1585
2051
986
1247
2311
3047
3223
2050
2271
370
2452
2439
43
337
2764
189
1648
3264
1199
3316
1386
295
1979
226
2973
1287
1807
2201
840
3018
161
3031
1428
2979
724
3238
1159
2968
2280
455
1403
2611
3318
2562
2429
2911
914
916
91
818
2223
642
3236
757
3322
3335
1496
3141
868
3034
1196
3261
1885
928
2019
188
3030
1543
1733
2714
3199
3163
400
2733
278
2353
2338
2900
1048
109
144
930
1802
14
2018
2766
1955
1435
2087
1338
2608
190
2180
2235
56
2103
1803
79
3095
1515
1696
2486
2117
2710
732
1808
154
405
2425
1931
245
1983
2684
2248
239
2368
3211
25
1574
3325
3338
1092
468
1188
2921
1

In [12]:
with open("tmp_gender.pickle", "wb") as f:
    pickle.dump((images,labels,gender), f)

In [13]:
with open("tmp_gender.pickle", "rb") as f:
    images, labels, gender = pickle.load(f) 

In [16]:
# img = cv2.imread('ankitanarula.jpg',0)
# resized = cv2.resize(img, (100, 100), interpolation = cv2.INTER_AREA)
# edge_images = cv2.Canny(resized,100,200)
# #edges = cv2.Canny(img,100,200)
# plt.subplot(121),plt.imshow(resized,cmap = 'gray')
# plt.title('Original Image'), plt.xticks([]), plt.yticks([])
# plt.subplot(122),plt.imshow(edge_images,cmap = 'gray')
# plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
# plt.show()
# edge_images

In [17]:
images[0].shape

(1, 20000)

## Making the Dataset 

In [19]:
res_images = []
res_gender = []

#for img in images:
#    res_images.append(img.reshape(1,20000))    # (100*100 image size)

for gen in gender:
    res_gender.append(np.array(gen).reshape(1,))
    
res_images = np.concatenate(images, axis=0)
res_gender = np.concatenate(res_gender,axis=0)

In [20]:
res_images.shape

(2908, 20000)

In [21]:
X,y = res_images,res_gender

## Test the Model for Gender with Edge

In [22]:
def predict(X,y):

    kf = KFold(X.shape[0], n_folds=10, shuffle=True)
    accuracy = 0.0
    precision = [0, 0]
    recall = [0, 0]
    f_measure = [0, 0]

    lr = LogisticRegression()
    inbuilt_accuracy_score = 0.0

    for test, train in kf:
        X_Train, X_Test = X[train], X[test]
        Y_Train, Y_Test = y[train], y[test]

        lr.fit(X_Train, Y_Train)

        pred = lr.predict(X_Test)

        inbuilt_accuracy_score += accuracy_score(Y_Test, (pred))

    #precision = map(lambda x: x / 10, precision)
    #recall = map(lambda x: x / 10, recall)
    #f_measure = map(lambda x: x / 10, f_measure)
    #accuracy /= 10
    #print "Confusion Matrix : \n\n", cm

    inbuilt_accuracy_score /= 10

    #print "*" * 100
    #print "\t\t\t Inbuilt Method"

    print "\nAccuracy from inbuilt Method: ", inbuilt_accuracy_score
    print(metrics.classification_report(Y_Test, pred))
    print(metrics.confusion_matrix(Y_Test, pred))

In [23]:
predict(X,y)


Accuracy from inbuilt Method:  0.652415554056
             precision    recall  f1-score   support

          0       0.66      0.60      0.63      1172
          1       0.70      0.75      0.72      1446

avg / total       0.68      0.68      0.68      2618

[[ 699  473]
 [ 363 1083]]


## Test the Model for Gender without Edge

In [24]:
X_without_edge,y = res_images[:,:10000],res_gender
X_without_edge.shape

(2908, 10000)

In [25]:
predict(X_without_edge,y)


Accuracy from inbuilt Method:  0.707817093559
             precision    recall  f1-score   support

          0       0.66      0.68      0.67      1163
          1       0.74      0.72      0.73      1455

avg / total       0.70      0.70      0.70      2618

[[ 796  367]
 [ 413 1042]]


## Sample of Edge

In [26]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('jeffweiner08.jpg',0)
edges = cv2.Canny(img,100,200)

plt.subplot(121),plt.imshow(img,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

plt.show()

In [6]:
#!/usr/bin/env python
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import collections
%matplotlib inline

#mu, sigma = 100, 15
#x = mu + sigma*np.random.randn(10000)
gender = [prof['Gender'] for prof in my_original_list]
#x= np.array(gender) 
x = collections.Counter(gender)
print x
# # the histogram of the data
# n, bins, patches = plt.hist(x, 50, normed=1, facecolor='blue', alpha=0.75)

# # add a 'best fit' line
# #y = mlab.normpdf(bins, mu, sigma)
# #l = plt.plot(bins, y, 'r--', linewidth=1)

# plt.xlabel('Age')
# plt.ylabel('Probability')
# plt.title(r'$Histogram\ \ of\ \ Ages$')
# #plt.axis([1, 80, 0, 0.09])
# plt.grid(True)

# plt.show()

Counter({'Male': 1458, 'Female': 1130, 'Unknown': 12})
